In [1]:
import json
import requests
import networkx as nx
import matplotlib.pyplot as plt

import plotly.plotly as py
import plotly.tools as tls
from plotly.graph_objs import *

mpl_fig = plt.figure()

In [2]:
personas = 'http://story-chronicles.herokuapp.com/storyobjects/'

target = requests.get(personas)

x = target.json()

In [3]:
story_objects = {}
labels = {}
node_colors = []

In [4]:
for character in x:
    name = character["name"].strip()
    story = character["story"]
    c_type = character["c_type"]
    story_objects[name] = {}
    story_objects[name]['name'] = name
    story_objects[name]['story'] = story
    story_objects[name]['c_type'] = c_type
    story_objects[name]['to_relationships'] = []
    if character['c_type'] == "Character":
        story_objects[name]['node_shape'] = 'o'
        story_objects[name]['node_color'] = 'r'
    elif character['c_type'] == "Organization":
        story_objects[name]['node_shape'] = 'h'
        story_objects[name]['node_color'] = 'b'
    elif character['c_type'] == "Creature":
        story_objects[name]['node_shape'] = '^'
        story_objects[name]['node_color'] = 'g'
    elif character['c_type'] == "Force":
        story_objects[name]['node_shape'] = 'v'
        story_objects[name]['node_color'] = 'c'
    elif character['c_type'] == "Thing":
        story_objects[name]['node_shape'] = 's'
        story_objects[name]['node_color'] = 'y'

    for relationship in character["to_relationships"]:
        break_1 = relationship.find(">>")
        break_2 = relationship.find("weight:")
        sub_1 = relationship[0:break_1].strip()
        context = relationship[break_1:break_2]
        weight = relationship[break_2+8:-1]
        story_objects[name]['to_relationships'].append([sub_1, context, weight])


In [5]:
G=nx.MultiDiGraph()

In [6]:
edge_labels = {}

for sub in story_objects:
    s = story_objects[sub]
    if s['story'] == "http://story-chronicles.herokuapp.com/story/1/":
        G.add_node(s['name'], node_shape=s['node_shape'])
        labels[s['name']] = s['name']
        
        #node_colors.append(s['node_color'])

        print("***", s['name'], "***", s['c_type'])
        print("details:", s['node_color'], s['node_shape'])
        for i in s['to_relationships']:
            print('target:', i[0])
            print('context:', i[1])
            print('weight:', i[2])
            G.add_edge(s['name'], i[0], weight=int(i[2]))
            edge_labels[(s['name'],i[0])] = i[1]
        print("")

*** Ivy *** Creature
details: g ^
target: Kaeso Messor
context: >> Familiar Ivy (A purple worm larvae) - 
weight: 5

*** Queen Mab *** Character
details: r o
target: Hoab
context: >> Studied magic under Queen Mab (A terrifying creature) - 
weight: 50

*** Hector *** Character
details: r o

*** Mammura of Formiae *** Character
details: r o

*** Yvarra *** Character
details: r o
target: Cogidubnus
context: >> Passionately Loves Yvarra (All-consuming) - 
weight: 99

*** Shadow's Kiss *** Thing
details: y s
target: Salvadar the Dwarf
context: >> Forged Shadow's Kiss (A gift in repayment for my life) - 
weight: 6

*** Salvadar the Dwarf *** Character
details: r o

*** Marcus Antonius *** Character
details: r o
target: Orphans
context: >> Serves Marcus Antonius (We owe him everything) - 
weight: 9
target: Kaeso Messor
context: >> Ally Marcus Antonius (Friend and supporter of Caesar) - 
weight: 3

*** Oblius *** Character
details: r o
target: Logos of Ios
context: >> Distrusts Oblius (Dangero

In [7]:
to_edge_labels = {}
from_edge_labels = {}
labels2 = {}

H = nx.MultiDiGraph()

def graph_so_relationships(so_node):
    
    # Add source node
    H.add_node(so_node)
    node_colors.append(story_objects[so_node]['node_color'])
    labels2[so_node] = so_node

    # Add nodes for all neighbours to source node
    for i in nx.neighbors(G, so_node):
        H.add_node(i)
        node_colors.append(story_objects[i]['node_color'])
        labels2[i] = i
        
        # Add one further level of relationships
        for n in story_objects[i]['to_relationships']:
            H.add_edge(story_objects[i]['name'], n[0], weight=int(n[2]))
            to_edge_labels[(story_objects[i]['name'],n[0])] = n[1]
            H.add_node(n[0])
            node_colors.append(story_objects[n[0]]['node_color'])
            labels2[n[0]] = n[0]
            
    # Add source node edges and labels from to_relationships
    for n in story_objects[so_node]['to_relationships']:
        H.add_edge(story_objects[so_node]['name'], n[0], weight=int(n[2]))
        from_edge_labels[(story_objects[so_node]['name'],n[0])] = n[1]


In [8]:
graph_so_relationships("Cogidubnus")

In [9]:
H.degree(weight=weight)
pos=nx.spring_layout(H)

In [10]:
nx.draw_networkx_nodes(H, pos, node_color=node_colors)
nx.draw_networkx_edges(H, pos, arrows=True)
nx.draw_networkx_labels(H, pos, labels2)
"""
nx.draw_networkx_edge_labels(H, pos, edge_labels=from_edge_labels, alpha=0.5,
                            font_color='r', label_pos=0.05)
nx.draw_networkx_edge_labels(H, pos, edge_labels=to_edge_labels, alpha=0.5,
                            font_color='b', label_pos=0.05)
"""

"\nnx.draw_networkx_edge_labels(H, pos, edge_labels=from_edge_labels, alpha=0.5,\n                            font_color='r', label_pos=0.05)\nnx.draw_networkx_edge_labels(H, pos, edge_labels=to_edge_labels, alpha=0.5,\n                            font_color='b', label_pos=0.05)\n"

In [11]:
plt.show()

In [18]:
nx.write_graphml(H,"test.graphml")

In [26]:
py.iplot_mpl(mpl_fig)


/Library/Frameworks/Python.framework/Versions/3.4/lib/python3.4/site-packages/plotly/matplotlylib/renderer.py:443: UserWarning:

Dang! That path collection is out of this world. I totally don't know what to do with it yet! Plotly can only import path collections linked to 'data' coordinates

